In [ ]:
!pip install pmdarima

In [1]:
# import the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
import math
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.stattools import adfuller, pacf, acf
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Data Preparation

In [2]:
# interval in minutes
interval = 10

In [ ]:
dec_data = pd.read_csv("anon_quik_dec_23.csv")
jan_data = pd.read_csv("anon_quik_jan_24.csv")
feb_data = pd.read_csv("anon_quik_feb_24.csv")

combined_data = pd.concat([dec_data, jan_data, feb_data], ignore_index=True)

combined_data['order_received_timestamp'] = pd.to_datetime(combined_data['order_received_timestamp'])
combined_data = combined_data.sort_values('order_received_timestamp')

merchant_ids = combined_data[['merchant_id']].drop_duplicates()
merchant_ids['seq_id'] = ['merchant' + str(i + 1) for i in range(len(merchant_ids))]
print(merchant_ids)

combined_data['Timestamp'] = combined_data['order_received_timestamp'].dt.floor(f'{interval}min')

demand_per_merchant = combined_data.groupby(['merchant_id', 'Timestamp']).size().reset_index(name='Orders')

demand_per_merchant = demand_per_merchant.merge(merchant_ids, on='merchant_id').drop(columns=['merchant_id'])
demand_per_merchant = demand_per_merchant.rename(columns={'seq_id': 'merchant_id'}).sort_values('Timestamp')

print(demand_per_merchant.describe())

dataframes = {merchant_id: df.drop(columns=['merchant_id']) for merchant_id, df in demand_per_merchant.groupby('merchant_id')}

def fill_missing_intervals(merchant_df, interval, start_date, end_date):
    full_timestamps = pd.date_range(start=start_date, end=end_date, freq=f'{interval}min')
    full_df = pd.DataFrame({'Timestamp': full_timestamps})

    merchant_df['Timestamp'] = pd.to_datetime(merchant_df['Timestamp'])
    full_df['Timestamp'] = pd.to_datetime(full_df['Timestamp'])

    merchant_df = pd.merge(full_df, merchant_df, on='Timestamp', how='left').fillna(0)

    return merchant_df

last_vals_dec = {}
last_vals_jan = {}

for i, (merchant_id, merchant_df) in enumerate(dataframes.items(), start=1):
    merchant_df = fill_missing_intervals(merchant_df, interval, "2023-12-01 00:10:00", "2024-02-29 23:50:00")

    last_order_dec = merchant_df.loc[merchant_df['Timestamp'] == "2023-12-31 23:50:00", 'Orders'].values[0] if not merchant_df.loc[merchant_df['Timestamp'] == "2023-12-31 23:50:00", 'Orders'].empty else 0
    last_order_jan = merchant_df.loc[merchant_df['Timestamp'] == "2024-01-31 23:50:00", 'Orders'].values[0] if not merchant_df.loc[merchant_df['Timestamp'] == "2024-01-31 23:50:00", 'Orders'].empty else 0
    last_order_feb = merchant_df.loc[merchant_df['Timestamp'] == "2024-02-29 23:50:00", 'Orders'].values[0] if not merchant_df.loc[merchant_df['Timestamp'] == "2024-02-29 23:50:00", 'Orders'].empty else 0

    # merchant_df['Orders'] = merchant_df['Orders'].shift(1, fill_value=0)

    merchant_df.loc[merchant_df['Timestamp'] == "2024-01-01 00:00:00", 'Orders'] = last_order_dec
    
    merchant_df.loc[merchant_df['Timestamp'] == "2024-02-01 00:00:00", 'Orders'] = last_order_jan

    last_row_march = pd.DataFrame({
        'Timestamp': ['2024-03-01 00:00:00'],
        'Orders': [last_order_feb]
    })
    
    merchant_df = pd.concat([merchant_df, last_row_march], ignore_index=True)
    
    merchant_df.to_csv(f"{merchant_id}_{interval}_quik.csv", index=False)

print("Data combined and processed successfully.")


## Time Series

In [19]:
merchant = "merchant7"

In [20]:
df = pd.read_csv(f'{merchant}_{interval}_quik.csv', index_col='Timestamp', parse_dates=True)

In [ ]:
df.head()

In [8]:
df['LogOrders'] = np.log(np.where(df['Orders'] == 0, 1, df['Orders']))

# Train-Test Split

In [22]:
new_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq=str(interval)+'min')
df = df.reindex(new_index)

Ntest = int(len(df) / 5)
train = df.iloc[:-Ntest]
test = df.iloc[-Ntest:]

# ACF and PACF

In [ ]:
plot_acf(df.Orders, lags=30, alpha=0.05)
plt.title('Autocorrelation')
plt.xlabel('Number of lags')
plt.ylabel('correlation')
plt.savefig(f'{merchant}_{interval}_acf_quik.png')

In [ ]:
acf_vals, confint = acf(df.Orders, nlags=len(df), alpha=0.05)
print("ACF Values:", acf_vals)
print("Confidence Intervals:", confint)

In [ ]:
plot_pacf(df.Orders, ax=plt.gca(), lags=20, alpha=0.05)
plt.title('Partial Autocorrelation')
plt.xlabel('Number of lags')
plt.ylabel('correlation')
plt.savefig(f'{merchant}_{interval}_pacf_quik.png')

In [ ]:
pacf_vals, confint = pacf(df.Orders, nlags=math.floor(len(df)/2 - 1), alpha=0.05)
print("PACF Values:", pacf_vals)
print("Confidence Intervals:", confint)

# SARIMA(p, d, q) x (P, D, Q, m)

## m

In [ ]:
from scipy.signal import find_peaks

fft_vals = np.fft.fft(df.Orders)
freqs = np.fft.fftfreq(len(fft_vals), d=1.0)

mags = np.abs(fft_vals)

pos_freqs = freqs[freqs > 0]
pos_mags = mags[freqs > 0]

peaks, _ = find_peaks(pos_mags, height=0.1 * np.max(pos_mags))

peak_freqs = pos_freqs[peaks]
peak_periods = 1 / peak_freqs

for i, peak_freq in enumerate(peak_freqs):
    print(peak_periods[i], pos_mags[peaks[i]])

plt.plot(1 / pos_freqs, pos_mags)
plt.xlim([0, 200])
plt.xlabel('Period')
plt.ylabel('Magnitude')
plt.title('Fourier Transform')
plt.savefig(f'{merchant}_{interval}_peaks_quik.png')

In [ ]:
peak_periods.sort()
m = round(peak_periods[-1])
print("m =", m)

In [ ]:
# seasonal period = 24 x 60 / 15 = 96
m = int(24 * 60 / interval)
print("m =", m)

In [33]:
seasonalities = np.delete(peak_periods, -1)
seasonalities = np.round(seasonalities)

## I(d)

In [ ]:
df['Orders'].plot();

## ADF

In [ ]:
adfuller(df['Orders'])

In [18]:
# first 2 return values are test-statistic and p-value
# https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html

In [36]:
def adf(x):
  res = adfuller(x)
  print("Test statistic:", res[0])
  print("P-value:", res[1])
  if res[1] < 0.05:
    print("Stationary")
  else:
    print("Non-Stationary")

In [ ]:
adf(df['Orders'])

In [ ]:
# by default the time series is assumed to be stationary
d = 0
res = adfuller(df['Orders'])


while res[1] > 0.05:
    d += 1
    df['Orders'] = df['Orders'].diff().dropna()
    res = adfuller(df['Orders'])

print("d =", d)

## KPSS

In [39]:
import statsmodels.api as sm
def kpss(x):
    res = sm.tsa.kpss(x, regression='ct') 
    print("Test statistic:", res[0])
    print("P-value:", res[1])
    if res[1] > 0.05:
        print("Stationary")
    else:
        print("Non-Stationary")

In [ ]:
kpss(df['Orders'])

## D

In [41]:
# time series has a steady seasonal pattern over time
D = 1

## AR(p)

In [ ]:
p = 0
for i in range(len(pacf_vals)):
  if (pacf_vals[i] - 2/np.sqrt(len(df['Orders']))) > 0:
    continue
  else:
    p = i-2
    break

print("p =", p)

## P

In [ ]:
P = []

if pacf_vals[m] > 0:
  P = [1, 2]
else:
  P = [0]

print("P =", P)

## Q

In [ ]:
Q = []

if acf_vals[m] > 0:
  Q = [0]
else:
  Q = [1, 2]

print("Q =", Q)

In [ ]:
plot_acf(df.Orders, lags=math.floor(len(df)/2 - 1), alpha=0.05);

## Seasonal Decomposition

In [ ]:
from statsmodels.tsa.seasonal import MSTL

res = MSTL(train['Orders'], periods=([48, 72])).fit()
res.plot()
plt.tight_layout()
plt.show()

In [80]:
trend = res.trend
seasonal = res.seasonal
residual = res.resid

# seasonals = pd.DataFrame()*2
# for i in range(len(seasonalities)):
#     seasonals[i] = 'seasonal_'+str(seasonalities[i])

exog = pd.concat([pd.DataFrame(['seasonal_48']), pd.DataFrame(['seasonal_72'])], axis=1)

In [ ]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# vif_data = pd.DataFrame()
# vif_data['Feature'] = exog.columns
# vif_data['VIF'] = [variance_inflation_factor(exog.values, i) for i in range(exog.shape[1])]
# print(vif_data)

In [ ]:
res_test = MSTL(test['Orders'], periods=(48, 72)).fit()
res_test.plot()
plt.tight_layout()
plt.show()

In [82]:
trend_test = res_test.trend
seasonal_test = res_test.seasonal
residual_test = res_test.resid

exog_test = pd.concat([pd.DataFrame(['seasonal_48']), pd.DataFrame(['seasonal_72'])], axis=1)

## Training

In [83]:
# def fourier_terms(t, period, order):
#     x = 2 * np.pi * (t / period)
#     return np.column_stack([np.cos(x * i) for i in range(1, order+1)] +
#                            [np.sin(x * i) for i in range(1, order+1)])

# t = np.arange(len(train["Orders"]))
# # fourier_32 = fourier_terms(t, 32, order=1)
# fourier_48 = fourier_terms(t, 48, order=2)
# # exog = np.column_stack((fourier_32, fourier_48))
# exog = fourier_48

In [ ]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# vif_data = pd.DataFrame()
# vif_data["VIF"] = [variance_inflation_factor(exog, i) for i in range(exog.shape[1])]
# vif_data["Feature"] = [f'Fourier_term_{i}' for i in range(exog.shape[1])]
# print(vif_data)
print(exog.dtypes)

In [ ]:
model = SARIMAX(train['Orders'],
                order=(p, d, 8),
                seasonal_order=(P[1], D, Q[0], m),
                exog=exog,
                trend='ct',
                enforce_stationarity=False,
                enforce_invertibility=True)

In [ ]:
results = model.fit(maxiter=1000)

In [ ]:
# def fourier_terms(t, period, order):
#     x = 2 * np.pi * (t / period)
#     return np.column_stack([np.cos(x * i) for i in range(1, order+1)] +
#                            [np.sin(x * i) for i in range(1, order+1)])

# t = np.arange(len(train["Orders"]))
# # fourier_32 = fourier_terms(t, 32, order=1)
# fourier_48 = fourier_terms(t, 48, order=2)
# # exog = np.column_stack((fourier_32, fourier_48))
# exog = fourier_48

In [ ]:
results.summary()

# Static Forecast

In [179]:
forecast = results.get_forecast(steps=Ntest, exog=exog_test)
test_pred = forecast.predicted_mean
confint = forecast.conf_int()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(test.index, test['Orders'], label='data')
ax.plot(test.index, test_pred, label='forecast')
ax.fill_between(test.index, \
                confint.iloc[:,0], confint.iloc[:,1], \
                color='red', alpha=0.3)
ax.legend();
plt.savefig(f'{merchant}_{interval}_out_of_sample_quik.png')

In [ ]:
train_pred = results.get_prediction(start=0, end=-1)

fig, ax = plt.subplots(figsize=(15, 6))
ax.plot(df.index, df['Orders'], label='data')
ax.plot(train.index, train_pred.predicted_mean, label='fitted')
ax.plot(test.index, test_pred, label='forecast')
ax.fill_between(test.index, \
                confint.iloc[:,0], confint.iloc[:,1], \
                color='red', alpha=0.3)
ax.legend();
plt.savefig(f'{merchant}_{interval}_quik.png')

# Log Transformation

In [ ]:
plot_acf(df.LogOrders, lags=50, alpha=0.05)
plt.title('Autocorrelation')
plt.xlabel('Number of lags')
plt.ylabel('correlation')
plt.savefig(f'{merchant}_{interval}_log_acf_quik.png')

In [ ]:
acf_vals, confint = acf(df.LogOrders, nlags=len(df), alpha=0.05)
print("ACF Values:", acf_vals)
print("Confidence Intervals:", confint)

In [ ]:
plot_pacf(df.LogOrders, ax=plt.gca(), lags=20, alpha=0.05)
plt.title('Partial Autocorrelation')
plt.xlabel('Number of lags')
plt.ylabel('correlation')
plt.savefig(f'{merchant}_{interval}_log_pacf_quik.png')

In [ ]:
df['LogOrders'].plot();

In [ ]:
pacf_vals, confint = pacf(df.LogOrders, nlags=math.floor(len(df)/2 - 1), alpha=0.05)
print("PACF Values:", pacf_vals)
print("Confidence Intervals:", confint)

In [ ]:
fft_vals = np.fft.fft(df.LogOrders)
freqs = np.fft.fftfreq(len(fft_vals), d=1.0)

mags = np.abs(fft_vals)

pos_freqs = freqs[freqs > 0]
pos_mags = mags[freqs > 0]

peak_idx = np.argmax(pos_mags)
peak_freq = pos_freqs[peak_idx]
peak_period = 1 / peak_freq if peak_freq != 0 else np.nan

print(f"Peak Period: {peak_period}")

plt.plot(1 / pos_freqs, pos_mags)
plt.xlim([0, 150])
plt.xlabel('Period')
plt.ylabel('Magnitude')
plt.title('Fourier Transform')
plt.savefig(f'{merchant}_{interval}_ft_quik.png')

In [28]:
m = round(peak_period)

In [ ]:
adfuller(df['LogOrders'])

In [ ]:
adf(df['LogOrders'])

In [ ]:
kpss(df['LogOrders'])

In [ ]:
p = 0
for i in range(len(pacf_vals)):
  if (pacf_vals[i] - 2/np.sqrt(len(df['Orders']))) > 0:
    continue
  else:
    p = i-2
    break

print("p =", p)

In [ ]:
P = []

if acf_vals[m] > 0:
  P = [1, 2]
else:
  P = [0]

print("P =", P)

In [ ]:
Q = []

if acf_vals[m] > 0:
  Q = [0]
else:
  Q = [1, 2]

print("Q =", Q)

In [90]:
d = 0
logmodel = SARIMAX(train['LogOrders'],
                order=(p, d, 8),
                seasonal_order=(P[1]+1, D, 1, m),
                enforce_stationarity=False,
                enforce_invertibility=True)

In [ ]:
results = logmodel.fit(maxiter=1000)

In [ ]:
results.summary()

In [63]:
forecast_log = results.get_forecast(steps=Ntest)
test_pred_log = forecast_log.predicted_mean
confint_log = forecast_log.conf_int()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(test.index, test['LogOrders'], label='data')
ax.plot(test.index, test_pred_log, label='forecast')
ax.fill_between(test.index, \
                confint_log.iloc[:,0], confint_log.iloc[:,1], \
                color='red', alpha=0.3)
ax.legend();
plt.savefig(f'{merchant}_{interval}_log_out_of_sample_quik.png')

In [ ]:
train_pred_log = results.get_prediction(start=0, end=-1)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df.index, df['LogOrders'], label='data')
ax.plot(train.index, train_pred_log.predicted_mean, label='fitted')
ax.plot(test.index, test_pred_log, label='forecast')
ax.fill_between(test.index, \
                confint_log.iloc[:,0], confint_log.iloc[:,1], \
                color='red', alpha=0.3)
ax.legend();
plt.savefig(f'{merchant}_{interval}_log_quik.png')

# Evaluation

## RMSE

In [182]:
### forecast RMSE
def rmse(t, y):
  return np.sqrt(np.mean(t - y)**2)

In [ ]:
print("Non-logged RMSE:", rmse(test['Orders'], test_pred))

In [ ]:
print("Logged RMSE:", rmse(test['Orders'], np.exp(test_pred_log)))

In [ ]:
print("Non-logged RMSE:", rmse(train['Orders'], train_pred.predicted_mean))

In [ ]:
print("Logged RMSE:", rmse(train['Orders'], np.exp(train_pred_log.predicted_mean)))

## MSE

In [153]:
### forecast MSE
def mse(t, y):
  return (np.mean(t - y)**2)

In [ ]:
print("Non-logged MSE:", mse(test['Orders'], test_pred))

In [ ]:
print("Logged MSE:", mse(test['Orders'], np.exp(test_pred_log)))

In [ ]:
print("Non-logged MSE:", mse(train['Orders'], train_pred.predicted_mean))

In [ ]:
print("Logged MSE:", mse(train['Orders'], np.exp(train_pred_log.predicted_mean)))

## MAE

In [156]:
### forecast MAE
def mae(t, y):
  return np.mean(np.abs(t - y))

In [ ]:
print("Non-logged MAE:", mae(test['Orders'], test_pred))

In [ ]:
print("Logged MAE:", mae(test['Orders'], np.exp(test_pred_log)))

In [ ]:
print("Non-logged MAE:", mae(train['Orders'], train_pred.predicted_mean))

In [ ]:
print("Logged MAE:", mae(train['Orders'], np.exp(train_pred_log.predicted_mean)))

## Theil's U-statistic

In [159]:
# forecast Theil's U-statistic
def ustatistic(t, y):
    num = np.sum((t - y)**2)
    den = np.sum((t - np.mean(t))**2)
    return np.sqrt(num / den)

In [ ]:
print("Non-logged Theil's U-statistic:", ustatistic(test['Orders'], test_pred))

In [ ]:
print("Logged Theil's U-statistic:", ustatistic(test['Orders'], np.exp(test_pred_log)))

In [ ]:
print("Non-logged Theil's U-statistic:", ustatistic(train['Orders'], train_pred.predicted_mean))

In [ ]:
print("Logged Theil's U-statistic:", ustatistic(train['Orders'], np.exp(train_pred_log.predicted_mean)))

# Residual Analysis

In [163]:
residuals = results.resid

In [ ]:
print(residuals.mean())

## Standardized Residuals

In [ ]:
plt.figure(figsize=(10, 6))
standardized_residuals = (residuals - residuals.mean()) / residuals.std()
plt.plot(standardized_residuals, label='Standardized Residuals')
plt.axhline(0, color='red', linestyle='--')
plt.title('Standardized Residuals')
plt.legend()
plt.savefig(f'{merchant}_{interval}_standardized_residuals_quik.png')

## Sample ACF of Residuals

In [ ]:
plt.figure(figsize=(12, 6))
plot_acf(residuals, lags=96, ax=plt.gca())
plt.title('Sample ACF of Residuals')
plt.xlabel('Number of lags')
plt.ylabel('correlation')
plt.savefig(f'{merchant}_{interval}_residuals_acf_quik.png')

## Normal Q-Q Plot

In [ ]:
import scipy.stats as stats
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Normal Q-Q Plot')
plt.savefig(f'{merchant}_{interval}_normal_qq_quik.png')

## Q-Statistic P-Values

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
ljung_box_test = acorr_ljungbox(residuals, lags=100, return_df=True)
print(ljung_box_test)

plt.plot(ljung_box_test.index, ljung_box_test['lb_pvalue'], marker='o', color='green')
plt.axhline(y=0.05, color='red', linestyle='--')
plt.title('Ljung-Box p-values')
plt.xlabel('Lag')
plt.ylabel('p-value')

In [ ]:
from scipy.stats import shapiro

stat, p_value = shapiro(residuals)
print(f'Shapiro-Wilk Test: Statistics={stat}, p-value={p_value}')

## Durbin-Watson Test

In [ ]:
from statsmodels.stats.stattools import durbin_watson
dw_statistic = durbin_watson(residuals)

print('Durbin-Watson statistic:', dw_statistic)

# Summary

In [171]:
filepath = f'{merchant}_{interval}_quik.txt'

with open(filepath, 'a') as file:
    file.write(results.summary().as_text())
    file.write('\n')
    file.write(f"Non-logged Test RMSE: {rmse(test['Orders'], test_pred)}\n")
    file.write(f"Non-logged Train RMSE: {rmse(train['Orders'], train_pred.predicted_mean)}\n")
    file.write('\n')
    file.write(f"Non-logged Test MSE: {mse(test['Orders'], test_pred)}\n")
    file.write(f"Non-logged Train MSE: {mse(train['Orders'], train_pred.predicted_mean)}\n")
    file.write('\n')
    file.write(f"Non-logged Test MAE: {mae(test['Orders'], test_pred)}\n")
    file.write(f"Non-logged Train MAE: {mae(train['Orders'], train_pred.predicted_mean)}\n")
    file.write('\n')
    file.write(f"Non-logged Test Theil's U-statistic: {ustatistic(test['Orders'], test_pred)}\n")
    file.write(f"Non-logged Train Theil's U-statistic: {ustatistic(train['Orders'], train_pred.predicted_mean)}\n")
    file.write('\n')
    file.write(f"Durban-Watson statistic: {dw_statistic}\n")